In [1]:
import numpy as np

In [2]:
# constants

P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6 ];
P8 =  [6, 3, 7, 4, 8, 5, 10, 9];

IP = [2, 6, 3, 1, 4, 8, 5, 7];
IP_INV = [4, 1, 3, 5, 7, 2, 8, 6];
EP = [4, 1, 2, 3, 2, 3, 4, 1];
P4 = [2, 4, 3, 1]

S0 = [ 
    [ 1, 0, 3, 2 ],
    [ 3, 2, 1, 0 ],
    [ 0, 2, 1, 3 ],
    [ 3, 1, 3, 2 ] 
]

S1 = [ 
    [ 0, 1, 2, 3 ],
    [ 2, 0, 1, 3 ],
    [ 3, 0, 1, 0 ],
    [ 2, 1, 0, 3 ] 
]

In [3]:
def permute(data, p):
    data = np.array(data)
    p = [i-1 for i in p]
    return list(data[p])

In [4]:
def left_shift(data, bits):
    return list(np.roll(data, -bits))

In [5]:
def generate_keys(key):
    p10 = permute(key, P10)
    lb = p10[:5]
    rb = p10[5:]
    
    lb1 = left_shift(lb, 1)
    rb1 = left_shift(rb, 1)
    key1 = permute(lb1+rb1, P8)
    
    lb12 = left_shift(lb1, 2)
    rb12 = left_shift(rb1, 2)
    key2 = permute(lb12+rb12, P8)
    
    return key1, key2

In [6]:
binary = {
    0: [0,0],
    1: [0,1],
    2: [1,0],
    3: [1,1],
}

def substitute(bits, box):
    r = 2*bits[0] + bits[3]
    c = 2*bits[1] + bits[2]
    d = box[r][c]
    return binary[d]

In [7]:
def swap(lb, rb):
    return rb, lb

In [8]:
def xor(a, b):
    a = np.array(a)
    b = np.array(b)
    return list(a^b)

In [9]:
def fk(data, key):
    lb = data[:4]
    rb = data[4:]
    
    r_exp = permute(rb, EP)
    r_xor = xor(r_exp, key)
    
    lb1 = r_xor[:4]
    rb1 = r_xor[4:]
    
    s0 = substitute(lb1, S0)
    s1 = substitute(rb1, S1)
    
    p4 = permute(s0+s1, P4)
    
    lb_new = xor(lb, p4)
    
    return lb_new, rb

In [10]:
def encrypt(data, key1, key2):
    p_init = permute(data, IP)
    lb1, rb1 = fk(p_init, key1)
    lb2, rb2 = swap(lb1, rb1)
    lb3, rb3 = fk(lb2+rb2, key2)
    p_inv = permute(lb3+rb3, IP_INV)
    return p_inv

In [11]:
def decrypt(data, key1, key2):
    p_init = permute(data, IP)
    lb1, rb1 = fk(p_init, key2)
    lb2, rb2 = swap(lb1, rb1)
    lb3, rb3 = fk(lb2+rb2, key1)
    p_inv = permute(lb3+rb3, IP_INV)
    return p_inv

<hr/>

In [12]:
# 1010000010
key = input("10 bit key: ")
key = [int(bit) for bit in key]
print('key:', key)

10 bit key: 1010000010
key: [1, 0, 1, 0, 0, 0, 0, 0, 1, 0]


In [13]:
# 10010111
message = input("8 bit message: ")
message = [int(bit) for bit in message]
print('message:', message)

8 bit message: 10010111
message: [1, 0, 0, 1, 0, 1, 1, 1]


In [14]:
key1, key2 = generate_keys(key)
print('key 1:', key1)
print('key 2:', key2)

key 1: [1, 0, 1, 0, 0, 1, 0, 0]
key 2: [0, 1, 0, 0, 0, 0, 1, 1]


In [15]:
cipher = encrypt(message, key1, key2)
print('cipher:', cipher)

cipher: [0, 0, 1, 1, 1, 0, 0, 0]


In [16]:
message = decrypt(cipher, key1, key2)
print('message:', message)

message: [1, 0, 0, 1, 0, 1, 1, 1]
